# Elections Tabulations

In [9]:
import pandas as pd
import re
import numpy as np
import math
from itertools import dropwhile
from collections import OrderedDict

import pyrankvote
from pyrankvote import Candidate, Ballot

In [63]:
raw_df = pd.read_csv("/Users/saruul/Desktop/Projects/asuc_ballot/2022ElectionResults.csv")
raw_df_trial = raw_df.replace('\xa0', ' ', regex=True)
raw_df_trial = raw_df_trial.replace(r"^ +| +$", r"", regex=True)
raw_df_trial['Transfer Representative - 1.2'].unique()


array([nan,
       'Scott Nagatoshi | Student Action | Amplifying Transfer Voices | ♥',
       'Aileen Sanchez | ElevateCal | ♥'], dtype=object)

In [64]:
raw_df_trial['Academic Affairs Vice President - 1.2'].unique()

array([nan, 'James Weichert | Independent | Current AAVP* | ♥',
       'None of the above'], dtype=object)

In [10]:
def fix_col_names(df_csv):
    df = df_csv.dropna(how = 'all').reset_index(drop = True)
    return df.add_suffix('.0')

def fix_non_break_space(df):
    return df.replace('\xa0', ' ', regex=True)

# Position:
# position = 'President'
def get_positional_data(position, raw_df_csv):
    # Fix Column names to avoid cols with ranks >= 10 getting deleted in the data cleanup
    ## col structure: 
    ## 'Senate - 10 - 10.0' where 10.0 is the indicator number for rank 10 (not 1 or anything else)
    raw_df = fix_non_break_space(raw_df_csv)
    raw_df = fix_col_names(raw_df)
    
    END_SUFFIX = '.0'
    SEPARATOR = ' - '
    pos_lst = ['President', 'Executive Vice President', 
                'External Affairs Vice President', 'Academic Affairs Vice President',
               'Student Advocate', 'Transfer Representative', 'Senate']
    # add suffix for columns about positions only
    raw_df.columns = [col + SEPARATOR + (re.findall(r'\d+', col)[0]) + END_SUFFIX if col.startswith(tuple(pos_lst)) else col for col in raw_df.columns] # raw_df.add_suffix('.0')

    # raw_df = raw_df.dropna(subset=raw_df.columns[0]).reset_index(drop = True)

    position_str = position + SEPARATOR
    president_cols = [col for col in raw_df if col.startswith(position)]
    pres_counts = 0

    while len(president_cols) != 0:
        pres_counts += 1
        pres_one_col = [col for col in raw_df if col.endswith(SEPARATOR + str(pres_counts) + END_SUFFIX)] # raw_df.filter(like = position_str + str(pres_counts) + '.') # 

        president_cols = [x for x in president_cols if x not in pres_one_col]

    pres_counts # Total number of pres selection options: President - 'pres_counts' is the last column name

    rslt_df = pd.DataFrame()
    pres_candidate_names = []
    for i in range(pres_counts):
        # i + 1 -> ranking in that position
        rank = str(i + 1)
        col_name = position_str + rank # + SEPARATOR + rank + END_SUFFIX
        end_name = str(i + 1) + END_SUFFIX
        
        # take account of duplicate columns for the same position & ranking
        pres_num_col = [col for col in raw_df if (col.startswith(position_str) & col.endswith(end_name))]
        pres_num_raw = raw_df[pres_num_col]
        
        # compress the rows to avoid repeating cols
        pres_final_num_df = pd.DataFrame(pres_num_raw.bfill(axis=1).iloc[:, 0])
        # there should only be one column df
        pres_final_num_df.columns = [col_name]
        
        # unique candidates in this column -> appended to all pres candidate lists
        unique_cand_subset = list(pres_final_num_df[col_name].unique())
        pres_candidate_names = pres_candidate_names + list(set(unique_cand_subset) - set(pres_candidate_names))

        # concat to the resulting df
        if rslt_df.empty:
            rslt_df = pres_final_num_df
        else:
            rslt_df = pd.concat([rslt_df, pres_final_num_df], axis=1).reset_index(drop=True)
    return rslt_df.dropna(axis = 0, how = 'all').reset_index(drop=True)



In [61]:
# PRESIDENTIAL CAMPAIGN
def exec_calculations(df):
#     works for president but not for other exec positions yet
    trial_elect_df = df
    df_cols = list(trial_elect_df.columns)
    len_df = len(trial_elect_df) * len(df_cols)
    MAX_RECURSION = 25
    # no_rslt_lst = ["No Response - " + str(i) for i in range(MAX_RECURSION)] * ((len_df // MAX_RECURSION) + 1)
    # for c in range(len(df_cols)):
    #     col_name = df_cols[c]
    #     trial_elect_df[col_name] = [no_rslt_lst[i] if type(v)==type(np.nan) else v for i,v in enumerate(trial_elect_df[col_name].tolist())]
    no_rslt_str = "No Response"
    for c in range(len(df_cols)):
        col_name = df_cols[c]
        # trial_elect_df[col_name] = [no_rslt_lst[i] if type(v)==type(np.nan) else v for i,v in enumerate(trial_elect_df[col_name].tolist())]
        trial_elect_df[col_name] = [no_rslt_str if type(v)==type(np.nan) else v for i,v in enumerate(trial_elect_df[col_name].tolist())]

    raw_cand_str_df = trial_elect_df # copying surface level
    raw_cand_str_df

    ## Assign Candidates
    unique_cands = pd.unique(trial_elect_df[list(trial_elect_df.columns)].values.ravel('K'))
    FIRST_CHAR = 6
    candidates = []
    cand_dict = {}
    unique_cands
    for cand in unique_cands:
        if (not pd.isnull(cand)) or (cand != ''):
            var_name_raw = "".join([s[0] for s in cand.split()][:FIRST_CHAR])
            var_name = re.sub('[^a-zA-Z]+', '', var_name_raw)
            vars()[var_name] = Candidate(cand)
            candidates.append(vars()[var_name])
            cand_dict[cand] = vars()[var_name]

    # Replace str name to Candidate value
    for col in raw_cand_str_df.columns:
        raw_cand_str_df[col] = raw_cand_str_df[col].map(cand_dict)

    # Add the results to Ballot
    ballots = []
    

    raw_cand_str_df = raw_cand_str_df.loc[raw_cand_str_df.nunique(axis=1).ne(1)].reset_index(drop = True)
    for i in range(len(raw_cand_str_df)):
        lst = raw_cand_str_df.loc[i, :].values.flatten().tolist()
        # Avoid any trailing NaN values
        res = list(reversed(tuple(dropwhile(lambda x: x is candidates[2],
                                        reversed(lst)))))
        rslt = Ballot(ranked_candidates = res)
        ballots.append(rslt)

    # Return result
    election_result = pyrankvote.instant_runoff_voting(candidates, ballots)
    winners = election_result.get_winners()

    return election_result

def get_final_rslt(election_result):
    all_rounds = election_result.__dict__["rounds"]
    return all_rounds[len(all_rounds) - 1].__str__()

position = "President"
rslt_df = get_positional_data(position, raw_df)
get_final_rslt(exec_calculations(rslt_df))

'Candidate                                                                      Votes  Status\n---------------------------------------------------------------------------  -------  --------\nChaka Tellem | ElevateCal | Current ASUC President | ♥                          5547  Elected\nCatherine Bauer | Student Action | Tenacious ideas, tangible solutions. | ♥     2953  Rejected\nNo Response                                                                       41  Rejected'

In [12]:
def senate_calculations(raw_df):
    df = get_positional_data("Senate", raw_df)
    trial_elect_df = df
    df_cols = list(trial_elect_df.columns)
    len_df = len(trial_elect_df) * len(df_cols)
    MAX_RECURSION = 25
    # no_rslt_lst = ["No Response - " + str(i) for i in range(MAX_RECURSION)] * ((len_df // MAX_RECURSION) + 1)
    no_rslt_str = "No Response"
    for c in range(len(df_cols)):
        col_name = df_cols[c]
        # trial_elect_df[col_name] = [no_rslt_lst[i] if type(v)==type(np.nan) else v for i,v in enumerate(trial_elect_df[col_name].tolist())]
        trial_elect_df[col_name] = [no_rslt_str if type(v)==type(np.nan) else v for i,v in enumerate(trial_elect_df[col_name].tolist())]

    raw_cand_str_df = trial_elect_df # copying surface level
    raw_cand_str_df

    ## Assign Candidates
    unique_cands = pd.unique(trial_elect_df[list(trial_elect_df.columns)].values.ravel('K'))
    FIRST_CHAR = 6
    candidates = []
    cand_dict = {}
    unique_cands
    for cand in unique_cands:
        if (not pd.isnull(cand)) or (cand != ''):
            var_name_raw = "".join([s[0] for s in cand.split()][:FIRST_CHAR])
            var_name = re.sub('[^a-zA-Z]+', '', var_name_raw)
            vars()[var_name] = Candidate(cand)
            candidates.append(vars()[var_name])
            cand_dict[cand] = vars()[var_name]
    # Replace str name to Candidate value
    for col in raw_cand_str_df.columns:
        raw_cand_str_df[col] = raw_cand_str_df[col].map(cand_dict)

    # Add the results to Ballot
    ballots = []


    raw_cand_str_df = raw_cand_str_df.loc[raw_cand_str_df.nunique(axis=1).ne(1)].reset_index(drop = True)
    for i in range(len(raw_cand_str_df)):
        lst = raw_cand_str_df.loc[i, :].values.flatten().tolist()

        target_element = Candidate("No Response")

        # delete trailing 'No Response' values
        def takewhile_including(iterable, value):
            for it in iterable:
                yield it
                if it == value:
                    return
        with_dup = list(takewhile_including(lst, target_element))[:-1]

        res = list(OrderedDict.fromkeys(with_dup))

        rslt = Ballot(ranked_candidates = res)
        ballots.append(rslt)

    # # Return result
    # election_result = pyrankvote.instant_runoff_voting(candidates, ballots)
    NUMBER_OF_SEATS = 20
    election_result = pyrankvote.single_transferable_vote(
        candidates, ballots, number_of_seats=20
    )
    winners = election_result.get_winners()
    return election_result
get_final_rslt(senate_calculations(raw_df))

'Candidate                                                                            Votes  Status\n---------------------------------------------------------------------------------  -------  --------\nCarlos C. Vasquez | ElevateCal | Co-Chair, Disabled Students Commission | ♥         380.00  Elected\nMegan Yao | Student Action | East Asian, International, Pre-Graduate | ♥            380.00  Elected\nStephanie Wong | ElevateCal | Asian American, SVSH, Housing, Reform | ♥             380.00  Elected\nEmma Centeno | Independent | Eco Community Endorsed | ♥                             380.00  Elected\nShay Cohen | Independent | Jewish Community, SVSH, Transparency | ♥                 380.00  Elected\nDeborah Kim | Independent | Christian, Mental Wellness, Com. Service | ♥            380.00  Elected\nShri Gopal | ElevateCal | Student Needs, STEM/Business, Greek Life | ♥              380.00  Elected\nAkash Ponna | Student Action | Dance, Wellness, Equity & Inclusion | ♥              380.00

In [44]:
# pos_lst = ['President', 'Executive Vice President', 
#                 'External Affairs Vice President', 'Academic Affairs Vice President',
#                'Student Advocate', 'Transfer Representative', 'Senate']
# Works for all Executive Positions!
position = 'President'
rslt_df = get_positional_data(position, raw_df)
get_final_rslt(exec_calculations(rslt_df))
# get_final_rslt(exec_calculations(get_positional_data(position, raw_df)))

'Candidate                                                                      Votes  Status\n---------------------------------------------------------------------------  -------  --------\nChaka Tellem | ElevateCal | Current ASUC President | ♥                          5547  Elected\nNo Response - 17                                                                   3  Rejected\nNo Response - 3                                                                    3  Rejected\nNo Response - 20                                                                   3  Rejected\nNo Response - 0                                                                    3  Rejected\nNo Response - 10                                                                   2  Rejected\nNo Response - 6                                                                    2  Rejected\nNo Response - 21                                                                   2  Rejected\nNo Response - 15                          

In [18]:
pos_lst = ['President', 'Executive Vice President', 
                'External Affairs Vice President', 'Academic Affairs Vice President',
               'Student Advocate', 'Transfer Representative', 'Senate']
submission_col = ["SubmissionId"]
# add suffix for columns about positions only
usable_df = raw_df.dropna(axis = 1, how = 'all')
prop_cols = [col for col in usable_df.columns if not col.startswith(tuple(pos_lst + submission_col)) ] # raw_df.add_suffix('.0')


In [14]:
def get_propositional_data(proposition, raw_df_csv):
    # Fix Column names to avoid cols with ranks >= 10 getting deleted in the data cleanup
    raw_df = fix_col_names(raw_df_csv)

    position_str = proposition
    president_cols = [col for col in raw_df if col.startswith(proposition)]

    pres_counts = len(president_cols) # Total number of pres selection options: President - 'pres_counts' is the last column name

    rslt_df = pd.DataFrame()
    pres_candidate_names = []
    for i in range(pres_counts):
        # i + 1 -> ranking in that position
        rank = str(i + 1)
        col_name = position_str + rank # + SEPARATOR + rank + END_SUFFIX
        
        # take account of duplicate columns for the same position & ranking
        pres_num_col = [col for col in raw_df if (col.startswith(position_str))]
        pres_num_raw = raw_df[pres_num_col]
        
        # compress the rows to avoid repeating cols
        pres_final_num_df = pd.DataFrame(pres_num_raw.bfill(axis=1).iloc[:, 0])
        # there should only be one column df
        pres_final_num_df.columns = [col_name]
        
        # unique candidates in this column -> appended to all pres candidate lists
        unique_cand_subset = list(pres_final_num_df[col_name].unique())
        pres_candidate_names = pres_candidate_names + list(set(unique_cand_subset) - set(pres_candidate_names))

        # concat to the resulting df
        if rslt_df.empty:
            rslt_df = pres_final_num_df
        else:
            rslt_df = pd.concat([rslt_df, pres_final_num_df], axis=1).reset_index(drop=True)
    return rslt_df.dropna(axis = 0, how = 'all').reset_index(drop=True)

In [43]:
# proposition_lst = ['Proposition 22A', 'Proposition 22B']
def proposition_calculation(proposition_name, raw_df):
    prop_trial_df = get_propositional_data(proposition_name, raw_df)
    prop_trial_df

    proposition_cols = list(prop_trial_df.columns)
    usage_df = pd.DataFrame()
    if len(proposition_cols) > 1:
        # all proposition columns have to the same
        prop_trial_df['matching'] = prop_trial_df.eq(prop_trial_df.iloc[:, 0], axis=0).all(1)
        matching_rows = prop_trial_df['matching'].unique()
        if ((len(matching_rows) == 1) & (matching_rows[0] == True)):
            print("All rows are the same")
            usage_df = prop_trial_df[[prop_trial_df.columns[0]]]
        else:
            # Assuming combining of cols are needed
            no_rslt_str = "No Response"
            for c in range(len(proposition_cols)):
                col_name = proposition_cols[c]
                prop_trial_df[col_name] = [no_rslt_str if type(v)==type(np.nan) else v for i,v in enumerate(prop_trial_df[col_name].tolist())]
    else:
        usage_df = prop_trial_df
    # usage_df is our main character
    usage_col = usage_df.columns[0]
    usage_df[usage_col] = usage_df[usage_col].str.strip()
    return usage_df[usage_col].value_counts().rename_axis('Votes').reset_index(name='counts')
proposition_calculation("Proposition 22B", raw_df)


All rows are the same


/var/folders/n5/syc1tyqx5w9g_5qswrg8c00c0000gn/T/ipykernel_27572/1109686091.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usage_df[usage_col] = usage_df[usage_col].str.strip()


,Votes,counts
0,Yes,1359
1,No,258
2,Abstain,200


In [39]:
trial = proposition_calculation("Proposition 22A", raw_df)
trial["Votes"].unique()

All rows are the same


array(['Yes', 'Abstain', 'No', 'No '], dtype=object)

In [28]:
prop_trial_df['matching'].unique()[0] == True

True

In [92]:
temp = []
for cand in pre_det_kst:
    row_list[row_list == vars()[cand]] = Candidate(vars()[cand])
    print(row_list)

[<Candidate('Catherine Bauer | Student Action | Tenacious ideas, tangible solutions. | ♥')>, nan]
[<Candidate('Chaka Tellem | ElevateCal | Current ASUC President | ♥')>, nan]
[<Candidate('nan')>, nan]


In [97]:
row_list[row_list == vars()["cand_1"]] = Candidate(vars()["cand_1"])

In [529]:
dict_trial = {
    "President - 1": ["George W. Bush (Republican)",
       "George W. Bush (Republican)",
       "George W. Bush (Republican)",
       "George W. Bush (Republican)",
        "Ralph Nader (Green)",
        "Ralph Nader (Green)",
        "Al Gore (Democratic)",
        "Al Gore (Democratic)",
        "Al Gore (Democratic)"
       ],
    "President - 2": [
        "Ralph Nader (Green)",
        "Ralph Nader (Green)",
        "Ralph Nader (Green)",
        "Ralph Nader (Green)",
        "Al Gore (Democratic)",
        "Al Gore (Democratic)",
        "Ralph Nader (Green)",
        "Ralph Nader (Green)",
        "Ralph Nader (Green)"
    ],
    "President - 3" : [
        "Al Gore (Democratic)",
        "Al Gore (Democratic)",
        float("nan"),
        float("nan"),
        "George W. Bush (Republican)",
        float("nan"),
        "George W. Bush (Republican)",
        float("nan"),
        float("nan"),
        
    ]
}
trial_elect_df = pd.DataFrame(dict_trial)


# PRESIDENTIAL CAMPAIGN
def pres_calculations(df):
    trial_elect_df = df
    df_cols = list(trial_elect_df.columns)
    len_df = len(trial_elect_df) * len(df_cols)
    MAX_RECURSION = 25
    no_rslt_lst = ["No Response - " + str(i) for i in range(MAX_RECURSION)] * ((len_df // MAX_RECURSION) + 1)
    for c in range(len(df_cols)):
        col_name = df_cols[c]
        trial_elect_df[col_name] = [no_rslt_lst[i] if type(v)==type(np.nan) else v for i,v in enumerate(trial_elect_df[col_name].tolist())]
    
    raw_cand_str_df = trial_elect_df # copying surface level
    raw_cand_str_df

    ## Assign Candidates
    unique_cands = pd.unique(trial_elect_df[list(trial_elect_df.columns)].values.ravel('K'))
    FIRST_CHAR = 6
    candidates = []
    cand_dict = {}
    unique_cands
    for cand in unique_cands:
        if (not pd.isnull(cand)) or (cand != ''):
            var_name_raw = "".join([s[0] for s in cand.split()][:FIRST_CHAR])
            var_name = re.sub('[^a-zA-Z]+', '', var_name_raw)
            vars()[var_name] = Candidate(cand)
            candidates.append(vars()[var_name])
            cand_dict[cand] = vars()[var_name]

    # Replace str name to Candidate value
    for col in raw_cand_str_df.columns:
        raw_cand_str_df[col] = raw_cand_str_df[col].map(cand_dict)

    # Add the results to Ballot
    ballots = []

    raw_cand_str_df = raw_cand_str_df.loc[raw_cand_str_df.nunique(axis=1).ne(1)].reset_index(drop = True)
    for i in range(len(raw_cand_str_df)):
        lst = raw_cand_str_df.loc[i, :].values.flatten().tolist()
        # Avoid any trailing NaN values
        res = list(reversed(tuple(dropwhile(lambda x: x is candidates[2],
                                        reversed(lst)))))
        rslt = Ballot(ranked_candidates = res)
        ballots.append(rslt)

    # Return result
    election_result = pyrankvote.instant_runoff_voting(candidates, ballots)
    winners = election_result.get_winners()

    return election_result

def get_final_rslt(election_result):
    all_rounds = results.__dict__["rounds"]
    return all_rounds[len(all_rounds) - 1].__str__()

In [492]:
concatted = pd.concat([pres_final_df, pres_final_two_df]).reset_index(drop=True)
results = pres_calculations(concatted)

In [530]:
get_final_rslt(results)

'Candidate                                                                      Votes  Status\n---------------------------------------------------------------------------  -------  --------\nChaka Tellem | ElevateCal | Current ASUC President | ♥                          7627  Elected\nNo Response - 1                                                                  276  Rejected\nNo Response - 11                                                                 273  Rejected\nCatherine Bauer | Student Action | Tenacious ideas, tangible solutions. | ♥     6513  Rejected\nNo Response - 3                                                                  277  Rejected\nNo Response - 23                                                                 272  Rejected\nNo Response - 10                                                                   0  Rejected\nNo Response - 18                                                                   0  Rejected\nNo Response - 22                          

In [518]:
help(results)

Help on class Candidate in module pyrankvote.models:

class Candidate(builtins.object)
 |  Candidate(name: str)
 |  
 |  A candidate in the election.
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, other) -> bool
 |      Return self==value.
 |  
 |  __hash__(self)
 |      Return hash(self).
 |  
 |  __init__(self, name: str)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self) -> str
 |      Return repr(self).
 |  
 |  __str__(self) -> str
 |      Return str(self).
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [494]:
print(results)
with open("pres_output.txt", "a") as f:
  print(results, file=f)
  # print("I have a question.", file=f)

ROUND 1
Candidate                                                                      Votes  Status
---------------------------------------------------------------------------  -------  --------
Chaka Tellem | ElevateCal | Current ASUC President | ♥                          5547  Hopeful
Catherine Bauer | Student Action | Tenacious ideas, tangible solutions. | ♥     2953  Hopeful
No Response - 19                                                                 291  Hopeful
No Response - 15                                                                 287  Hopeful
No Response - 0                                                                  282  Hopeful
No Response - 20                                                                 281  Hopeful
No Response - 22                                                                 280  Hopeful
No Response - 10                                                                 278  Hopeful
No Response - 3                                     

In [487]:
# import numpy as np
# dict_trial = {
#     "President - 1": ["George W. Bush (Republican)",
#        "George W. Bush (Republican)",
#        "George W. Bush (Republican)",
#        "George W. Bush (Republican)",
#         "Ralph Nader (Green)",
#         "Ralph Nader (Green)",
#         "Al Gore (Democratic)",
#         "Al Gore (Democratic)",
#         "Al Gore (Democratic)"
#        ],
#     "President - 2": [
#         "Ralph Nader (Green)",
#         "Ralph Nader (Green)",
#         "Ralph Nader (Green)",
#         "Ralph Nader (Green)",
#         "Al Gore (Democratic)",
#         "Al Gore (Democratic)",
#         "Ralph Nader (Green)",
#         "Ralph Nader (Green)",
#         "Ralph Nader (Green)"
#     ],
#     "President - 3" : [
#         "Al Gore (Democratic)",
#         "Al Gore (Democratic)",
#         float("nan"),
#         float("nan"),
#         "George W. Bush (Republican)",
#         float("nan"),
#         "George W. Bush (Republican)",
#         float("nan"),
#         float("nan"),
        
#     ]
# }
# trial_elect_df = pd.DataFrame(dict_trial)

# Adding No Response as cand
# PRESIDENTIAL
concatted = pd.concat([pres_final_df, pres_final_two_df]).reset_index(drop=True)
trial_elect_df = concatted
df_cols = list(trial_elect_df.columns)
len_df = len(trial_elect_df) * len(df_cols)
MAX_RECURSION = 25
no_rslt_lst = ["No Response - " + str(i) for i in range(MAX_RECURSION)] * ((len_df // MAX_RECURSION) + 1)
for c in range(len(df_cols)):
    col_name = df_cols[c]
    trial_elect_df[col_name] = [no_rslt_lst[i] if type(v)==type(np.nan) else v for i,v in enumerate(trial_elect_df[col_name].tolist())]
# col_name = "President - 1"
# trial_elect_df[col_name] = [no_rslt_lst[(0) * len(trial_elect_df) + i] if type(v)==type(np.nan) else v for i,v in enumerate(trial_elect_df[col_name].tolist())]

raw_cand_str_df = trial_elect_df # copying surface level
raw_cand_str_df

## Assign Candidates
unique_cands = pd.unique(trial_elect_df[list(trial_elect_df.columns)].values.ravel('K'))
FIRST_CHAR = 6
candidates = []
cand_dict = {}
unique_cands
for cand in unique_cands:
    if (not pd.isnull(cand)) or (cand != ''):
        var_name_raw = "".join([s[0] for s in cand.split()][:FIRST_CHAR])
        var_name = re.sub('[^a-zA-Z]+', '', var_name_raw)
        vars()[var_name] = Candidate(cand)
        candidates.append(vars()[var_name])
        cand_dict[cand] = vars()[var_name]
candidates

# Replace str name to Candidate value
for col in raw_cand_str_df.columns:
    raw_cand_str_df[col] = raw_cand_str_df[col].map(cand_dict)
raw_cand_str_df

# Add the results to Ballot
ballots = []

raw_cand_str_df = raw_cand_str_df.loc[raw_cand_str_df.nunique(axis=1).ne(1)].reset_index(drop = True)
for i in range(len(raw_cand_str_df)):
    lst = raw_cand_str_df.loc[i, :].values.flatten().tolist()
    res = list(reversed(tuple(dropwhile(lambda x: x is candidates[2],
                                    reversed(lst)))))
    rslt = Ballot(ranked_candidates = res)
    ballots.append(rslt)
ballots

# Return result
election_result = pyrankvote.instant_runoff_voting(candidates, ballots)
election_result
winners = election_result.get_winners()
# # Returns: [<Candidate('Al Gore (Democratic)')>]

print(election_result)

ROUND 1
Candidate                                                                      Votes  Status
---------------------------------------------------------------------------  -------  --------
Chaka Tellem | ElevateCal | Current ASUC President | ♥                          5547  Hopeful
Catherine Bauer | Student Action | Tenacious ideas, tangible solutions. | ♥     2953  Hopeful
No Response - 19                                                                 291  Hopeful
No Response - 15                                                                 287  Hopeful
No Response - 0                                                                  282  Hopeful
No Response - 20                                                                 281  Hopeful
No Response - 22                                                                 280  Hopeful
No Response - 10                                                                 278  Hopeful
No Response - 3                                     

In [488]:
winners

[<Candidate('Chaka Tellem | ElevateCal | Current ASUC President | ♥')>]

In [480]:
df = pd.DataFrame({
         'B':[4,4,4,4,4,4],
         'C':[4,4,9,4,2,3],
         'D':[4,4,5,7,1,0],

})

# print (df)
#    B  C  D
# 0  4  4  4
# 1  4  4  4
# 2  4  9  5
# 3  4  4  7
# 4  4  2  1
# 5  4  3  0

df2 = df.loc[df.nunique(axis=1).ne(1)]
df2

,B,C,D
2,4,9,5
3,4,4,7
4,4,2,1
5,4,3,0


In [443]:
s.loc[:s.first_valid_index()] = 0

0

In [465]:
concatted = pd.concat([pres_final_df, pres_final_two_df]).reset_index(drop=True)
concatted.columns

Index(['President - 1', 'President - 2'], dtype='object')

In [368]:
df_cols = list(trial_elect_df.columns)
len_df = len(trial_elect_df) * len(df_cols)
no_rslt_lst = ["'No Result - " + str(i) for i in range(len_df + 1)]
for c in range(len(df_cols)):
    col_name = df_cols[c]
    trial_elect_df[col_name] = [no_rslt_lst[(c) * len(trial_elect_df) + i] if type(v)==type(np.nan) else v for i,v in enumerate(trial_elect_df[col_name].tolist())]


In [450]:
trial_elect_df

,President - 1,President - 2
0,Catherine Bauer | Student Action | Tenacious i...,'No Response - 21822
1,Chaka Tellem | ElevateCal | Current ASUC Presi...,'No Response - 21823
2,Catherine Bauer | Student Action | Tenacious i...,'No Response - 21824
3,Catherine Bauer | Student Action | Tenacious i...,'No Response - 21825
4,'No Response - 4,'No Response - 21826
...,...,...
21817,'No Response - 21817,'No Response - 43639
21818,'No Response - 21818,Catherine Bauer | Student Action | Tenacious i...
21819,'No Response - 21819,Catherine Bauer | Student Action | Tenacious i...
21820,'No Response - 21820,Catherine Bauer | Student Action | Tenacious i...


In [347]:

[temp_lst[i] if type(v)==type(np.nan) else v for i,v in enumerate(trial_elect_df['President - 3'].tolist())]



['Al Gore (Democratic)',
 'Al Gore (Democratic)',
 "'No Result - 2",
 "'No Result - 3",
 'George W. Bush (Republican)',
 "'No Result - 5",
 'George W. Bush (Republican)',
 "'No Result - 7",
 "'No Result - 8"]

In [342]:
for i,v in enumerate(trial_elect_df['President - 3'].tolist()):
    print(i)

0
1
2
3
4
5
6
7
8


In [293]:
len(ballots)

15238

In [309]:
trial = concatted.dropna(how = "all")
trial = trial.fillna(0)
trial[trial.eq(0)] = ''
trial.to_csv("trial.csv")

In [310]:
trial

,President - 1,President - 2
0,Catherine Bauer | Student Action | Tenacious i...,
1,Chaka Tellem | ElevateCal | Current ASUC Presi...,
2,Catherine Bauer | Student Action | Tenacious i...,
3,Catherine Bauer | Student Action | Tenacious i...,
6,Chaka Tellem | ElevateCal | Current ASUC Presi...,
...,...,...
21816,,Chaka Tellem | ElevateCal | Current ASUC Presi...
21818,,Catherine Bauer | Student Action | Tenacious i...
21819,,Catherine Bauer | Student Action | Tenacious i...
21820,,Catherine Bauer | Student Action | Tenacious i...


In [313]:
trial = raw_cand_str_df.loc[0, :].values.flatten().tolist()
list(reversed(tuple(dropwhile(lambda x: x is candidates[2],
                                    reversed(lst)))))

[<Candidate('Catherine Bauer | Student Action | Tenacious ideas, tangible solutions. | ♥')>,
 <Candidate('No Response')>]

In [315]:

ballots = []
for i in range(len(raw_cand_str_df)):
    lst = raw_cand_str_df.loc[i, :].values.flatten().tolist()
    res = list(reversed(tuple(dropwhile(lambda x: x is candidates[2],
                                    reversed(lst)))))
    rslt = Ballot(ranked_candidates = res)
    ballots.append(rslt)
ballots

[<Ballot(Catherine Bauer | Student Action | Tenacious ideas, tangible solutions. | ♥)>,
 <Ballot(Chaka Tellem | ElevateCal | Current ASUC President | ♥)>,
 <Ballot(Catherine Bauer | Student Action | Tenacious ideas, tangible solutions. | ♥)>,
 <Ballot(Catherine Bauer | Student Action | Tenacious ideas, tangible solutions. | ♥)>,
 <Ballot(Chaka Tellem | ElevateCal | Current ASUC President | ♥)>,
 <Ballot(Catherine Bauer | Student Action | Tenacious ideas, tangible solutions. | ♥)>,
 <Ballot(Chaka Tellem | ElevateCal | Current ASUC President | ♥)>,
 <Ballot(Catherine Bauer | Student Action | Tenacious ideas, tangible solutions. | ♥)>,
 <Ballot(Chaka Tellem | ElevateCal | Current ASUC President | ♥)>,
 <Ballot(Catherine Bauer | Student Action | Tenacious ideas, tangible solutions. | ♥)>,
 <Ballot(Chaka Tellem | ElevateCal | Current ASUC President | ♥)>,
 <Ballot(Chaka Tellem | ElevateCal | Current ASUC President | ♥)>,
 <Ballot(Catherine Bauer | Student Action | Tenacious ideas, tangible 

In [308]:
candidates[2]

<Candidate('No Response')>

,President - 1,President - 2,President - 3
0,George W. Bush (Republican),Ralph Nader (Green),Al Gore (Democratic)
1,George W. Bush (Republican),Ralph Nader (Green),Al Gore (Democratic)
2,George W. Bush (Republican),Ralph Nader (Green),No Response
3,George W. Bush (Republican),Ralph Nader (Green),No Response
4,Ralph Nader (Green),Al Gore (Democratic),George W. Bush (Republican)
5,Ralph Nader (Green),Al Gore (Democratic),No Response
6,Al Gore (Democratic),Ralph Nader (Green),George W. Bush (Republican)
7,Al Gore (Democratic),Ralph Nader (Green),No Response
8,Al Gore (Democratic),Ralph Nader (Green),No Response


ROUND 1
Candidate                      Votes  Status
---------------------------  -------  --------
George W. Bush (Republican)        4  Hopeful
Al Gore (Democratic)               3  Hopeful
Ralph Nader (Green)                2  Rejected
No Response                        0  Rejected

FINAL RESULT
Candidate                      Votes  Status
---------------------------  -------  --------
Al Gore (Democratic)               5  Elected
George W. Bush (Republican)        4  Rejected
Ralph Nader (Green)                0  Rejected
No Response                        0  Rejected



In [76]:
import pyrankvote
from pyrankvote import Candidate, Ballot

bush = Candidate("George W. Bush (Republican)")
gore = Candidate("Al Gore (Democratic)")
nader = Candidate("Ralph Nader (Green)")

candidates = [bush, gore, nader]

# Bush have most first choice votes, but because Ralph Nader-voters want
# Al Gore if Nader is not elected, the elected candidate is Al Gore
ballots = [
    Ballot(ranked_candidates=[bush, nader, gore]),
    Ballot(ranked_candidates=[bush, nader, gore]),
    Ballot(ranked_candidates=[bush, nader]),
    Ballot(ranked_candidates=[bush, nader]),
    Ballot(ranked_candidates=[nader, gore, bush]),
    Ballot(ranked_candidates=[nader, gore]),
    Ballot(ranked_candidates=[gore, nader, bush]),
    Ballot(ranked_candidates=[gore, nader]),
    Ballot(ranked_candidates=[gore, nader])
]

# You can use your own Candidate and Ballot objects as long as they implement the same properties and methods
election_result = pyrankvote.instant_runoff_voting(candidates, ballots)

winners = election_result.get_winners()
# Returns: [<Candidate('Al Gore (Democratic)')>]

print(election_result)

ROUND 1
Candidate                      Votes  Status
---------------------------  -------  --------
George W. Bush (Republican)        4  Hopeful
Al Gore (Democratic)               3  Hopeful
Ralph Nader (Green)                2  Rejected

FINAL RESULT
Candidate                      Votes  Status
---------------------------  -------  --------
Al Gore (Democratic)               5  Elected
George W. Bush (Republican)        4  Rejected
Ralph Nader (Green)                0  Rejected



In [64]:
'Catherine Bauer | Student Action | Tenacious ideas, tangible solutions. | ♥'

'Catherine Bauer | Student Action | Tenacious ideas, tangible solutions. | ♥'

In [52]:
bush = Candidate("George W. Bush (Republican)")
gore = Candidate("Al Gore (Democratic)")
nader = Candidate("Ralph Nader (Green)")

In [54]:
print(bush)

George W. Bush (Republican)


In [11]:
list(raw_df.columns)

['SubmissionId',
 'The Save the Daily Cal Initiative,a $6 per semester and $2.50 per summer fee assessed to all undergraduate and graduate students,including students enrolled in self-supporting graduate professional degree programs,would provide The Daily Californian with the financial support necessary to (1) continue production of The Daily Californian’s free content,(2) continue printing and distributing free,independent publications of The Daily Californian across campus and surrounding areas at least weekly,(3) maintain a platform for UC Berkeley students to engage with writing,reporting,graphic and web design and business operations opportunities,(4) expand and maintain its online digital media and news coverage and (5) increasing opportunities for UC Berkeley student journalists through scholarships and financial aid. The Save the Daily Cal Initiative,subject to an annual inflation adjustment starting in the 2023-2024 academic year,and a flat summer fee of $2.50,would last for 

In [10]:
raw_df.columns

Index(['SubmissionId',
       'The Save the Daily Cal Initiative,a $6 per semester and $2.50 per summer fee assessed to all undergraduate and graduate students,including students enrolled in self-supporting graduate professional degree programs,would provide The Daily Californian with the financial support necessary to (1) continue production of The Daily Californian’s free content,(2) continue printing and distributing free,independent publications of The Daily Californian across campus and surrounding areas at least weekly,(3) maintain a platform for UC Berkeley students to engage with writing,reporting,graphic and web design and business operations opportunities,(4) expand and maintain its online digital media and news coverage and (5) increasing opportunities for UC Berkeley student journalists through scholarships and financial aid. The Save the Daily Cal Initiative,subject to an annual inflation adjustment starting in the 2023-2024 academic year,and a flat summer fee of $2.50,wou

In [417]:
# Pres 1
pres_one_col = [col for col in raw_df if col.startswith('President - 1')]
pres_one_raw = raw_df[pres_one_col]
pres_final_df = pd.DataFrame(pres_one_raw.bfill(axis=1).iloc[:, 0])
pres_lst = list(pres_final_df["President - 1"].unique())

# Pres 2
pres_two_col = [col for col in raw_df if col.startswith('President - 2')]
pres_two_raw = raw_df[pres_two_col]
pres_final_two_df = pd.DataFrame(pres_two_raw.bfill(axis=1).iloc[:, 0])
pres_two_lst = list(pres_final_two_df["President - 2"].unique())

concatted = pd.concat([pres_final_df, pres_final_two_df]).reset_index(drop=True)

['Catherine Bauer | Student Action | Tenacious ideas, tangible solutions. | ♥',
 'Chaka Tellem | ElevateCal | Current ASUC President | ♥',
 nan]

In [628]:
# col structure: 
# 'Senate - 10 - 10.0' where 10.0 is the indicator number for rank 10 (not 1 or anything else)
END_SUFFIX = '.0'
SEPARATOR = ' - '
pos_lst = ['President', 'Executive Vice President', 
            'External Affairs Vice President', 'Academic Affairs Vice President',
           'Student Advocate', 'Transfer Representative', 'Senate']
# add suffix for columns about positions only
raw_df.columns = [col + SEPARATOR + (re.findall(r'\d+', col)[0]) + END_SUFFIX if col.startswith(tuple(pos_lst)) else col for col in raw_df.columns] # raw_df.add_suffix('.0')


position_str = position + SEPARATOR
president_cols = [col for col in raw_df if col.startswith(position)]
pres_counts = 0

while len(president_cols) != 0:
    pres_counts += 1
    pres_one_col = [col for col in raw_df if col.endswith(SEPARATOR + str(pres_counts) + END_SUFFIX)] # raw_df.filter(like = position_str + str(pres_counts) + '.') # 

    president_cols = [x for x in president_cols if x not in pres_one_col]
pres_counts

32